In [1]:
import sys 
import os 
import numpy as np
import pandas as pd
import sklearn as sk
import re
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE
from sklearn import svm
from sklearn import metrics as metrics_sk
from sklearn.metrics import precision_recall_curve
from scipy.stats import ks_2samp
from IPython.display import Image, display_svg, SVG

In [2]:
from keras import layers, Sequential
from keras.layers import Input, Add, Dropout, Dense, Activation, ZeroPadding1D, ZeroPadding2D, BatchNormalization, Flatten, Conv1D, AveragePooling1D, MaxPooling1D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras import metrics
from keras import optimizers
from keras import regularizers
import pydot
from tensorflow.keras import regularizers
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
import tensorflow as tf
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.
C:\Anaconda3\envs\benedito_deep\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\envs\benedito_deep\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\envs\benedito_deep\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\envs\benedito_deep\lib\site-packages\tensorflo

In [3]:
from keras.backend import sigmoid
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

In [4]:
from keras.backend import log
def inv_h_sin(x, theta = 1):
    if x == 0:
        return x
    else:
        return log(theta*x + ((theta*x)**2 + 1)**0.5)/theta

In [5]:
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})
get_custom_objects().update({'inv_h_sin': Activation(inv_h_sin)})

In [6]:
os.getcwd()

'C:\\Users\\Benedito'

In [7]:
pd.set_option('display.max_columns', None)

In [8]:
os.chdir("C:/Users/Benedito/Desktop/MECAI/Introdução a Estatistica/MVP")

In [9]:
os.getcwd()

'C:\\Users\\Benedito\\Desktop\\MECAI\\Introdução a Estatistica\\MVP'

In [10]:
df = pd.read_csv("modelling_table_4.csv")

In [11]:
data = df

In [12]:
data.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_other,REGION_RATING_CLIENT_2,REGION_RATING_CLIENT_3,REGION_RATING_CLIENT_W_CITY_2,REGION_RATING_CLIENT_W_CITY_3,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_other,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes,cnt_appl_previous,cnt_revolving_previous,cnt_cash_previous,cnt_consumer_previous,avg_amt_annuity_previous,avg_amt_application_previous,avg_amt_credit_previous,avg_amt_down_payment_previous,avg_amt_goods_price_previous,cnt_appl_start_bus_day_previous,cnt_appl_start_weekend_previous,cnt_appl_start_dawn_p

In [13]:
data.fillna(0, inplace = True)

In [14]:
data.set_index('SK_ID_CURR', inplace=True)

In [15]:
data = data.replace([np.inf, -np.inf], 0)

In [16]:
np.random.seed(11235)
sample = data.sample(200000, random_state = 11235)

In [17]:
valid = data[~data.index.isin(sample.index)].sample(100000, random_state = 11235)
test_final = data[~((data.index.isin(valid.index))|data.index.isin(sample.index))]

In [18]:
X = sample.drop(['TARGET'], axis =1)
y = sample['TARGET']

In [19]:
X.fillna(0, inplace = True)

In [20]:
#smote = SMOTE(sampling_strategy = 0.1, random_state = 11235) ##0.15 fica bom

In [20]:
X_, y_ = X, y

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.3, random_state=1234)

In [ ]:
#from keras.callbacks import EarlyStopping

In [ ]:
#callback = EarlyStopping(monitor='AUC', patience=5)

## Neural Network

In [22]:
### MELHOR DE TODAS
NN_Model = Sequential()
NN_Model.add(Dense(32, input_dim=X_train.shape[-1], activation='swish'))
NN_Model.add(Dropout(0.5,seed = 11235))
NN_Model.add(Dense(256, activation='swish'))
NN_Model.add(Dropout(0.5,seed = 11235))
NN_Model.add(Dense(32, activation='swish'))
NN_Model.add(Dropout(0.5, seed = 11235))
NN_Model.add(Dense(1, activation='sigmoid'))

In [23]:
metrics_ = [metrics.FalseNegatives(name="fn"),metrics.FalsePositives(name="fp"),
           metrics.TrueNegatives(name="tn"),
           metrics.TruePositives(name="tp"),
           metrics.Precision(name="precision"),
           metrics.Recall(name="recall"),
           metrics.AUC(name="AUC"), 
           metrics.Accuracy(name="accuracy")]

In [26]:
#opt = optimizers.Nadam()

In [24]:
NN_Model.compile(loss='binary_crossentropy', optimizer="adam", metrics=metrics_)
#tf.keras.metrics.Precision(), tf.keras.metrics.Recall() tf.keras.metrics.AUC()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [25]:
class_weight = {0: 1/(y_train==0).sum(), 1: 1/(y_train==1).sum()}

In [26]:
scaler = StandardScaler()

In [27]:
feats = X_train.columns
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_valid = scaler.transform(valid[feats])
y_valid = valid['TARGET']
X_test_final = scaler.transform(test_final[feats])
y_test_final = test_final['TARGET']

In [28]:
NN_Model.fit(x = X_train, y = y_train, epochs = 128, batch_size = 2048, class_weight = class_weight)#, callbacks = [callback])


Epoch 1/128
140000/140000 [==============================] - 86s 617us/step - loss: 1.0615e-05 - fn: 6245.0000 - fp: 54768.0000 - tn: 73953.0000 - tp: 5034.0000 - precision: 0.0842 - recall: 0.4463 - AUC: 0.5152 - accuracy: 6.4286e-05
Epoch 2/128
140000/140000 [==============================] - 5s 36us/step - loss: 1.0169e-05 - fn: 4467.0000 - fp: 68208.0000 - tn: 60513.0000 - tp: 6812.0000 - precision: 0.0908 - recall: 0.6040 - AUC: 0.5469 - accuracy: 2.8571e-05
Epoch 3/128
140000/140000 [==============================] - 5s 34us/step - loss: 9.8934e-06 - fn: 4258.0000 - fp: 63954.0000 - tn: 64767.0000 - tp: 7021.0000 - precision: 0.0989 - recall: 0.6225 - AUC: 0.5880 - accuracy: 3.5714e-05
Epoch 4/128
140000/140000 [==============================] - 5s 37us/step - loss: 9.6485e-06 - fn: 4010.0000 - fp: 58353.0000 - tn: 70368.0000 - tp: 7269.0000 - precision: 0.1108 - recall: 0.6445 - AUC: 0.6279 - accuracy: 2.1429e-05: 1s - loss: 9.6570e-06 - fn: 2637.0000 - fp: 39328.0000 - tn: 454

In [29]:
predictions = NN_Model.predict(X_valid)
predictions_ = (predictions > 0.5).astype(int)

In [30]:
acc_train = metrics_sk.accuracy_score(y_train, (NN_Model.predict(X_train)>0.5))
f1_train = metrics_sk.f1_score(y_train, (NN_Model.predict(X_train)>0.5))
precision_train = metrics_sk.precision_score(y_train, (NN_Model.predict(X_train)>0.5))
recall_train = metrics_sk.recall_score(y_train, (NN_Model.predict(X_train)>0.5))
auc_train = metrics_sk.roc_auc_score(y_train, NN_Model.predict(X_train))

print('accuracy_train: ',acc_train)
print('precision_train: ',precision_train)
print('recall_train: ',recall_train)
print('F1_train: ',f1_train)
print('AUC_train: ',auc_train)

accuracy_train:  0.7099857142857143
precision_train:  0.18198959687906371
recall_train:  0.7409344800070928
F1_train:  0.2911284574329376
AUC_train:  0.7960679814258218


In [31]:
acc_test = metrics_sk.accuracy_score(y_test, (NN_Model.predict(X_test)>0.5))
f1_test = metrics_sk.f1_score(y_test, (NN_Model.predict(X_test)>0.5))
precision_test = metrics_sk.precision_score(y_test, (NN_Model.predict(X_test)>0.5))
recall_test = metrics_sk.recall_score(y_test, (NN_Model.predict(X_test)>0.5))
auc_test = metrics_sk.roc_auc_score(y_test, NN_Model.predict(X_test))

print('accuracy_test: ',acc_test)
print('precision_test: ',precision_test)
print('recall_test: ',recall_test)
print('F1_test: ',f1_test)
print('AUC_test: ',auc_test)

accuracy_test:  0.7062
precision_test:  0.17654534363562424
recall_test:  0.7005533920885427
F1_test:  0.27780933062880325
AUC_test:  0.7669678701490495


In [32]:
acc_valid = metrics_sk.accuracy_score(y_valid, (NN_Model.predict(X_valid)>0.5))
f1_valid = metrics_sk.f1_score(y_valid, (NN_Model.predict(X_valid)>0.5))
precision_valid = metrics_sk.precision_score(y_valid, (NN_Model.predict(X_valid)>0.5))
recall_valid = metrics_sk.recall_score(y_valid, (NN_Model.predict(X_valid)>0.5))
auc_valid = metrics_sk.roc_auc_score(y_valid, NN_Model.predict(X_valid))

print('accuracy_valid: ',acc_valid)
print('precision_valid: ',precision_valid)
print('recall_valid: ',recall_valid)
print('F1_valid: ',f1_valid)
print('AUC_valid: ',auc_valid)

accuracy_valid:  0.70388
precision_valid:  0.17273530576800025
recall_valid:  0.7020458772473651
F1_valid:  0.2775905027114172
AUC_valid:  0.7691019076626654


## LightGBM

In [ ]:
#dtree = LGBMClassifier(max_depth = 2, min_samples_split=100, min_data_in_leaf = 250,n_estimators=300,
#                            min_impurity_decrease=1e-03, lambda_l1=0.3,
#                       lambda_l2=0.7, random_state=11235, eta = 0.005)

In [ ]:
###GRID SEARCH LGBM ---> NAO RODAR SEMPRE

list_i = [2, 4, 6]
list_j = [50, 100, 200]
list_k = [200, 300, 400]
list_l = [100, 200, 300]

for i in list_i:
    for j in list_j:
        for k in list_k:
            for l in list_l:
                dtree = LGBMClassifier(max_depth = i, min_samples_split=j, min_data_in_leaf = k,n_estimators=l,
                            min_impurity_decrease=1e-03, lambda_l1=0.3, lambda_l2=0.7, random_state=11235, eta = 0.005)
                class_tree = dtree.fit(X_train, y_train)
                auc_valid = metrics_sk.roc_auc_score(y_valid, class_tree.predict_proba(X_valid)[:,1])
                print('depth: ', i, 'samples_split: ', j, 'data_in_leaf: ', k, 'n_estimators: ', l, 'auc_valid: ', auc_valid)

In [33]:
dtree = LGBMClassifier(max_depth = 4, min_samples_split=50, min_data_in_leaf = 400,n_estimators=300,
                            min_impurity_decrease=1e-03, lambda_l1=0.3,
                       lambda_l2=0.7, random_state=11235, eta = 0.005)

In [34]:
class_tree = dtree.fit(X_train, y_train)

In [35]:
acc_train = metrics_sk.accuracy_score(y_train, class_tree.predict(X_train))
f1_train = metrics_sk.f1_score(y_train, class_tree.predict(X_train))
precision_train = metrics_sk.precision_score(y_train, class_tree.predict(X_train))
recall_train = metrics_sk.recall_score(y_train, class_tree.predict(X_train))
auc_train = metrics_sk.roc_auc_score(y_train, class_tree.predict_proba(X_train)[:,1])

print('accuracy_train: ',acc_train)
print('precision_train: ',precision_train)
print('recall_train: ',recall_train)
print('F1_train: ',f1_train)
print('AUC_train: ',auc_train)

accuracy_train:  0.9228214285714286
precision_train:  0.7542918454935622
recall_train:  0.062328220586931465
F1_train:  0.11514208500532308
AUC_train:  0.8409009689035088


In [36]:
acc_test = metrics_sk.accuracy_score(y_test, class_tree.predict(X_test))
f1_test = metrics_sk.f1_score(y_test, class_tree.predict(X_test))
precision_test = metrics_sk.precision_score(y_test, class_tree.predict(X_test))
recall_test = metrics_sk.recall_score(y_test, class_tree.predict(X_test))
auc_test = metrics_sk.roc_auc_score(y_test, class_tree.predict_proba(X_test)[:,1])

print('accuracy_test: ',acc_test)
print('precision_test: ',precision_test)
print('recall_test: ',recall_test)
print('F1_test: ',f1_test)
print('AUC_test: ',auc_test)

accuracy_test:  0.9194
precision_test:  0.5609065155807366
recall_test:  0.040582086493133836
F1_test:  0.0756880733944954
AUC_test:  0.7891603461484586


In [37]:
acc_valid = metrics_sk.accuracy_score(y_valid, class_tree.predict(X_valid))
f1_valid = metrics_sk.f1_score(y_valid, class_tree.predict(X_valid))
precision_valid = metrics_sk.precision_score(y_valid, class_tree.predict(X_valid))
recall_valid = metrics_sk.recall_score(y_valid, class_tree.predict(X_valid))
auc_valid = metrics_sk.roc_auc_score(y_valid, class_tree.predict_proba(X_valid)[:,1])

print('accuracy_valid: ',acc_valid)
print('precision_valid: ',precision_valid)
print('recall_valid: ',recall_valid)
print('F1_valid: ',f1_valid)
print('AUC_valid: ',auc_valid)

accuracy_valid:  0.92034
precision_valid:  0.5774647887323944
recall_valid:  0.045753254804711714
F1_valid:  0.08478860294117646
AUC_valid:  0.7893968605207776


## Stacking NN+LGBM

In [38]:
pred_nn = NN_Model.predict(X_valid)
pred_lgbm = class_tree.predict_proba(X_valid)[:,1].reshape(-1, 1)


In [39]:
X_stack = np.zeros((pred_nn.shape[0], 2))

In [40]:
X_stack[:,0] = pred_nn.reshape(1, -1)
X_stack[:,1] = pred_lgbm.reshape(1, -1)
#X_stack[:,2] = pred_nn.reshape(1, -1)**2
#X_stack[:,3] = pred_lgbm.reshape(1, -1)**2
#X_stack[:,4] = pred_nn.reshape(1, -1)**3
#X_stack[:,5] = pred_lgbm.reshape(1, -1)**3

In [41]:
class_weight = {0: 1/(y_valid==0).sum(), 1: 1/(y_valid==1).sum()}

In [42]:
lr = LogisticRegression()#class_weight=class_weight

In [43]:
from sklearn.naive_bayes import GaussianNB

In [44]:
nb = GaussianNB(var_smoothing = 0.01)

In [45]:
lr_ = lr.fit(X_stack, y_valid)

In [46]:
nb_ = nb.fit(X_stack, y_valid)

## Validação - supervisor logistico

In [47]:
acc_stack = metrics_sk.accuracy_score(y_valid, lr_.predict(X_stack))
f1_stack = metrics_sk.f1_score(y_valid, lr_.predict(X_stack))
precision_stack = metrics_sk.precision_score(y_valid, lr_.predict(X_stack))
recall_stack = metrics_sk.recall_score(y_valid, lr_.predict(X_stack))
auc_stack = metrics_sk.roc_auc_score(y_valid, lr_.predict_proba(X_stack)[:,1])

print('accuracy_stack: ',acc_stack)
print('precision_stack: ',precision_stack)
print('recall_stack: ',recall_stack)
print('F1_stack: ',f1_stack)
print('AUC_stack: ',auc_stack)

accuracy_stack:  0.92024
precision_stack:  0.5438423645320197
recall_stack:  0.06844389336639801
F1_stack:  0.12158590308370044
AUC_stack:  0.7823893717733873


## Validação - supervisor Naive Bayes

In [48]:
acc_stack = metrics_sk.accuracy_score(y_valid, nb_.predict(X_stack))
f1_stack = metrics_sk.f1_score(y_valid, nb_.predict(X_stack))
precision_stack = metrics_sk.precision_score(y_valid, nb_.predict(X_stack))
recall_stack = metrics_sk.recall_score(y_valid, nb_.predict(X_stack))
auc_stack = metrics_sk.roc_auc_score(y_valid, nb_.predict_proba(X_stack)[:,1])

print('accuracy_stack: ',acc_stack)
print('precision_stack: ',precision_stack)
print('recall_stack: ',recall_stack)
print('F1_stack: ',f1_stack)
print('AUC_stack: ',auc_stack)

accuracy_stack:  0.89569
precision_stack:  0.33243626062322945
recall_stack:  0.29101053936763793
F1_stack:  0.3103471074380165
AUC_stack:  0.7790572040523929


In [49]:
pred_nn_final = NN_Model.predict(X_test_final)
pred_lgbm_final = class_tree.predict_proba(X_test_final)[:,1].reshape(-1, 1)

In [50]:
X_stack_final = np.zeros((pred_nn_final.shape[0], 2))

In [51]:
X_stack_final[:,0] = pred_nn_final.reshape(1, -1)
X_stack_final[:,1] = pred_lgbm_final.reshape(1, -1)
#X_stack_final[:,2] = pred_nn_final.reshape(1, -1)**2
#X_stack_final[:,3] = pred_lgbm_final.reshape(1, -1)**2
#X_stack_final[:,4] = pred_nn_final.reshape(1, -1)**3
#X_stack_final[:,5] = pred_lgbm_final.reshape(1, -1)**3

## Teste final - supervisor Naive Bayes

In [56]:
acc_stack = metrics_sk.accuracy_score(y_test_final, nb_.predict(X_stack_final))
f1_stack = metrics_sk.f1_score(y_test_final, nb_.predict(X_stack_final))
precision_stack = metrics_sk.precision_score(y_test_final, nb_.predict(X_stack_final))
recall_stack = metrics_sk.recall_score(y_test_final, nb_.predict(X_stack_final))
auc_stack = metrics_sk.roc_auc_score(y_test_final, nb_.predict_proba(X_stack_final)[:,1])

print('accuracy_stack: ',acc_stack)
print('precision_stack: ',precision_stack)
print('recall_stack: ',recall_stack)
print('F1_stack: ',f1_stack)
print('AUC_stack: ',auc_stack)

accuracy_stack:  0.8908267873785115
precision_stack:  0.3039568345323741
recall_stack:  0.2807308970099668
F1_stack:  0.2918825561312608
AUC_stack:  0.7841298532560688


## Teste final - supervisor logístico

In [57]:
acc_stack = metrics_sk.accuracy_score(y_test_final, lr_.predict(X_stack_final))
f1_stack = metrics_sk.f1_score(y_test_final, lr_.predict(X_stack_final))
precision_stack = metrics_sk.precision_score(y_test_final, lr_.predict(X_stack_final))
recall_stack = metrics_sk.recall_score(y_test_final, lr_.predict(X_stack_final))
auc_stack = metrics_sk.roc_auc_score(y_test_final, lr_.predict_proba(X_stack_final)[:,1])

print('accuracy_stack: ',acc_stack)
print('precision_stack: ',precision_stack)
print('recall_stack: ',recall_stack)
print('F1_stack: ',f1_stack)
print('AUC_stack: ',auc_stack)

accuracy_stack:  0.9197177473039542
precision_stack:  0.49230769230769234
recall_stack:  0.053156146179401995
F1_stack:  0.095952023988006
AUC_stack:  0.7857984361483337


# Optimizing F1 Score

In [64]:
#### optimizing f1 score
X_opt = np.zeros((X_stack.shape[0], 4))

In [65]:
X_opt[:,0] = 1
X_opt[:,1:3]=X_stack
X_opt[:,3] = y_valid

In [238]:
#def sigmoid(x): resulta em overflow essa merda
#    return 1/(1+np.exp(-x))

In [66]:
def sigmoid(x): ##corrigido
    if x.all()>=0:
        z = np.exp(-x)
        return 1/(1+z)
    else:
        z = np.exp(-x)
        return z/(1+z)

In [67]:
def sigmoid1(x): ##derivada
    return sigmoid(x)*(1-sigmoid(x))

In [68]:
alpha = 0.25 ##obtive via grid search 
gamma = 2

In [69]:
def f(theta, alpha = alpha,X_opt = X_opt, gamma = gamma):
    A = np.sum(sigmoid(gamma*np.dot(X_opt[X_opt[:,3]==1,:3], theta)))
    npos = X_opt[:,3].sum()
    mpos = np.sum(sigmoid(gamma*np.dot(X_opt[:,:3], theta)))
    F = A/(alpha*npos +(1-alpha)*mpos)
    return -F

In [70]:
def df(theta, alpha = alpha,X_opt = X_opt, gamma = gamma):
    A = np.sum(sigmoid(gamma*np.dot(X_opt[X_opt[:,3]==1,:3], theta)))
    npos = X_opt[:,3].sum()
    mpos = np.sum(sigmoid(gamma*np.dot(X_opt[:,:3], theta)))
    h = 1/(alpha*npos + (1-alpha)*mpos)
    dF=np.zeros((theta.shape))
    dA = np.zeros((theta.shape))
    dmpos = np.zeros((theta.shape))
    
    for j in range(X_opt.shape[1]-1):
        dA[j] = np.sum(sigmoid1(gamma*np.dot(X_opt[X_opt[:,3]==1,:3], theta))*gamma*X_opt[X_opt[:,3]==1,j])
        dmpos[j] = np.sum(sigmoid1(gamma*np.dot(X_opt[:,:3], theta))*gamma*X_opt[:,j])
        dF[j] = h*dA[j]-(h**2)*A*(1-alpha)*dmpos[j]
    return -dF
        
    

In [72]:
from scipy import optimize

In [74]:
def predict_opt(X, theta_opt = theta_opt):
    score = sigmoid(np.dot(X[:,:-1], theta_opt))
    predict = (score>0.5).astype(int)
    return (score, predict)

In [77]:
##GRID SEARCH + RANDOM INITIALIZATION --> nao rodar sempre
#np.random.seed(42)
#theta_matrix = np.random.randint(low = -21, high = 21, size = (20, 3)).round(2)

#list_alpha = [0.25, 0.5, 0.75] 
#list_gamma = [2, 4, 8, 16, 32]

#for i in range(len(list_alpha)):
#    for j in range(len(list_gamma)):
#        for k in range(theta_matrix.shape[0]):
#            
#            alpha = list_alpha[i]
#            gamma = list_gamma[j]
#            theta = theta_matrix[k,:].T
#            theta_opt = optimize.minimize(f, theta, jac = df, method = 'bfgs').x
#            #theta_opt,k = bfgs_method(f, df, theta, maxiter = None, epsi = 0.0001)
#            score, predicted = predict_opt(X_opt, theta_opt)
#            print('alpha = ', alpha, 'gamma = ', gamma, 'theta_init = ', theta,
#                  'theta_opt = ', theta_opt,
#                  'AUC = ', metrics_sk.roc_auc_score(y_valid, score),
#                  'precision = ', metrics_sk.precision_score(y_valid, predicted),
#                  'recall = ', metrics_sk.recall_score(y_valid, predicted),
#                  'F1 = ', metrics_sk.f1_score(y_valid, predicted))

alpha =  0.25 gamma =  2 theta_init =  [10  9 -4] theta_opt =  [10.  9. -4.] AUC =  0.7494770109518669 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha = 

C:\Anaconda3\envs\benedito_deep\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


 0.25 gamma =  2 theta_init =  [-19  -8 -12] theta_opt =  [-19.  -8. -12.] AUC =  0.21950093382171038 precision =  0.0 recall =  0.0 F1 =  0.0


C:\Anaconda3\envs\benedito_deep\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


alpha =  0.25 gamma =  2 theta_init =  [-15 -21   3] theta_opt =  [-15. -21.   3.] AUC =  0.23731329491634212 precision =  0.0 recall =  0.0 F1 =  0.0


...

C:\Anaconda3\envs\benedito_deep\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


alpha =  0.5 gamma =  4 theta_init =  [-2 -2  7] theta_opt =  [-7025.73350828  4525.59981372 12916.09378086] AUC =  0.626147769231415 precision =  0.3717873258779674 recall =  0.23496590204587725 F1 =  0.28795015955022035


C:\Anaconda3\envs\benedito_deep\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


alpha =  0.5 gamma =  4 theta_init =  [-14   3  18] theta_opt =  [-2173.31313553   859.46589014  5747.95634353] AUC =  0.6856390800921336 precision =  0.362094395280236 recall =  0.243521388716677 F1 =  0.29120023723033583
alpha =  0.5 gamma =  4 theta_init =  [ 13 -17  -5] theta_opt =  [ 21.17089098 -11.18920194  -3.34983505] AUC =  0.22832308508218174 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914


KeyboardInterrupt: 

In [ ]:
#alpha =  0.25 gamma =  2 theta_init =  [-1 -4  9] theta_opt =  [-568.50174834  322.85256065 2247.08499949] AUC =  0.7829109328873998 precision =  0.2642267969778887 recall =  0.4726596404215747 F1 =  0.338964965321003
#alpha =  0.25 gamma =  2 theta_init =  [0 3 5] theta_opt =  [-431.15928105  220.73593698 1731.39766752] AUC =  0.7840137316349044 precision =  0.26989136817381093 recall =  0.45592064476131433 F1 =  0.3390658859329614

In [78]:
alpha = 0.25
gamma = 2
theta_opt = np.array([-431.15928105,  220.73593698, 1731.39766752])

In [79]:
score, predicted = predict_opt(X_opt, theta_opt)

### Performance em validação (usado para otimizar)

In [82]:
print('AUC:', metrics_sk.roc_auc_score(y_valid, score))
print('PRECISION: ', metrics_sk.precision_score(y_valid, predicted))
print('RECALL: ', metrics_sk.recall_score(y_valid, predicted))
print('F1: ', metrics_sk.f1_score(y_valid, predicted))

AUC: 0.7839042733196057
PRECISION:  0.2683998535335042
RECALL:  0.45443273403595785
F1:  0.3374769797421731


In [83]:
X_opt_final = np.zeros((X_stack_final.shape[0], 4))
X_opt_final[:,0] = 1
X_opt_final[:,1:3]=X_stack_final
X_opt_final[:,3] = y_test_final

In [84]:
theta_opt = np.array([-431.15928105,  220.73593698, 1731.39766752]).T

In [85]:
score_final, predicted_final = predict_opt(X_opt_final, theta_opt)

In [86]:
X_opt_final.shape

(7511, 4)

In [89]:
X_stack_final.shape

(7511, 2)

### Performance no teste final

In [88]:
print('AUC', metrics_sk.roc_auc_score(y_test_final, score_final))
print('PRECISION', metrics_sk.precision_score(y_test_final, predicted_final))
print('RECALL', metrics_sk.recall_score(y_test_final, predicted_final))
print('F1', metrics_sk.f1_score(y_test_final, predicted_final))

AUC 0.7867188014670066
PRECISION 0.255859375
RECALL 0.43521594684385384
F1 0.3222632226322263
